In [84]:
# -*- coding: UTF-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import os

import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import sys
sys.path.append("..")
#from data.dataset.BatchGenerator import *

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import math

'''
读入一支股票指定年份的ohlcv数据
输入:baseDir,stockCode为字符, startYear,yearNum为整数，
输出:dataframe
'''
def readWSDFile(baseDir, stockCode, startYear, yearNum=1):
    # 解析日期
    filename = baseDir+stockCode+'/'+stockCode+'.csv'
    print (filename, "===============")
    dateparse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d').date()
    df = pd.read_csv(filename, index_col=0, sep=',', header=None,
                            skiprows=1, names=['date','pre_close','open','high','low','close','change','chg_range',
                                               'volume','amount','turn'],
                           parse_dates=True, date_parser=dateparse)
    return df['2005-01-04':'2015-12-31']

'''
读入一支股票指定年份的技术指标
输入:baseDir,stockCode为字符, startYear,yearNum为整数，
输出:dataframe
'''
def readWSDIndexFile(baseDir, stockCode, startYear, yearNum=1):
    # 解析日期
    dateparse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d').date()

    df = 0
    for i in range(yearNum):
        tempDF = pd.read_csv(baseDir+'I'+stockCode+'/wsd_'+stockCode+'_'+str(startYear+i)+'.csv', index_col=0, sep=',', parse_dates=True, date_parser=dateparse
                             # , usecols=usecols
                             )
        if i==0: df = tempDF
        else: df = df.append(tempDF)
    return df

# 整理好多因子输入，以dataframe返回数据+标签
from sklearn import preprocessing
def data_prepare(retrace = 0.618):
    # prepare data
    baseDir = '../data/'
    stockCodes = ['000300.SH']
    i = 0
    startYear = 2005
    number =11
    df = readWSDFile(baseDir, stockCodes[i], startYear, number)
    dfi = readWSDIndexFile(baseDir, stockCodes[i], startYear, number)
    allDF = pd.concat([df, dfi], axis=1)
    sample_num = np.shape(df)[0]
    labelDF = pd.Series(np.zeros(sample_num))
    print ("Factors Shape:", np.shape(df), np.shape(dfi))
    
    # 求出 trend
    price = df['close']
    start = 0
    while price[start] > price[start+1]:
        labelDF[start] = 1 #flat
        start +=1
    print("----- start: ",start)
    #find peak, find trough, calculate retracement and label trend accordingly
    i = start
    while i < sample_num - 1:
        cursor = i
        while cursor < sample_num - 1 and price[cursor] <= price[cursor+1]:
            cursor += 1
        peak = cursor
        while cursor < sample_num - 1 and price[cursor] >= price[cursor+1]:
            cursor += 1
        trough = cursor
        retracement = (price[peak] - price[trough]) / (price[peak] - price[i])
        mark = 1 # flat
        if retracement < retrace:
            mark = 2 # UP
        elif retracement > 1 + retrace:
            mark = 0 # DOWN
        for k in range(i, cursor+1):
            labelDF[k] = mark
        i = cursor

    print("---- Trend Distribution Check --------")
    print(labelDF.value_counts().sort_index())
    
    # make a deep copy of Price Difference before normalizing
    priceDF = allDF['change'].copy(deep=True)
    # normalize
    scaler = preprocessing.MinMaxScaler()
    input_data = scaler.fit_transform(allDF)
    
    print ("input data shape: ", np.shape(allDF)) #  days *  factors
    print ("input label shape: ", np.shape(labelDF))
   
    return allDF, labelDF, priceDF # train/test data, labels and prices for yield calucluation

def _data_prepare(file_name, retrace = 0.618, no_retrace = False, col=1):
    print(col, "===")
    dataset = pd.read_csv(file_name,index_col=0, sep=',', usecols=[0, col], skiprows=1, names=['date','close'],parse_dates=True)
    print(file_name)
    # calculate momentum: Mt = (CLOSE(t) -CLOSE(t-1))/CLOSE(t-1)
    dataset['mmt'] = 0.0
    dataset['diff'] = 0.0
    for i in range(1, len(dataset)):
        dataset['mmt'][i] = (dataset['close'][i] - dataset['close'][i-1]) / dataset['close'][i-1]
        dataset['diff'][i-1] = dataset['close'][i] - dataset['close'][i-1]
    
    pd.set_option('mode.chained_assignment',None)
    # classify by counts
    dataset['label'] = 0
    mmt_series = dataset['mmt']
    for i in range(len(dataset)):
        mmt = mmt_series[i]
        if mmt < -0.01:
            dataset['label'][i] = 0
        elif mmt <= 0.01:
            dataset['label'][i] = 1
        else:
            dataset['label'][i] = 2
    dataset['trend']=0
    price = dataset['close']
    size = len(price)
    start = 0
    while price[start] > price[start+1]:
        start +=1
    print("----- start: ",start)
    #find peak, find trough, calculate retracement and label trend accordingly
    i = start
    while i < size - 1:
        cursor = i
        while cursor < size - 1 and price[cursor] <= price[cursor+1]:
            cursor += 1
        peak = cursor
        while cursor < size - 1 and price[cursor] >= price[cursor+1]:
            cursor += 1
        trough = cursor
        retracement = (price[peak] - price[trough]) / (price[peak] - price[i])
        mark = 1 # flat
        if retracement < retrace:
            mark = 2 # UP
        elif retracement > 1 + retrace:
            mark = 0 # DOWN
        for k in range(i, cursor+1):
            dataset['trend'][k] = mark
        i = cursor

    print("---- Trend Distribution Check --------")
    print(dataset['trend'].value_counts().sort_index())
    print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) , " ------ Complete Data Preparation")
    #dataset['label'].plot(lw=2.0)
    #dataset.plot(lw=2.0)
    #plt.show()
    print("total data: ", len(dataset))
    if no_retrace == True:
        #print('whole data for SVM')
        return dataset['label']
    else:
        return dataset #dataset['trend'] # only need 1 series => 为了计算收益率，需要每日收盘价

###############
# NOTICE:
# We use rolling data here due to the small size of data, label is 1 step further into the future
# eg:
# batch0: X=[0,1,2,3] y=[4]
# batch1: X=[1,2,3,4] y=[5]

class BatchGenerator(object):
    def __init__(self, file_name, batch_size, train_ratio, time_steps, input_size, column = 1, retrace = 0.618, fold_i=0, use_weight=False, no_retrace=False, fine_grained=False):
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.time_steps = time_steps
        self.input_size = input_size
        self.no_retrace = no_retrace
        self.fine_grained = fine_grained
        #self.dataset = _data_prepare(file_name, retrace, no_retrace=no_retrace, col = column)
        self.dataset, self.labels, self.price_diff = data_prepare(retrace)
         # 为计算收益率，后一天的价差，预测涨则加
        self.segment_num = (len(self.dataset) - time_steps - 1) // batch_size # rollingly use data
        self.train_size = int(self.segment_num * train_ratio)
        self.test_size = self.segment_num - self.train_size
        
        self.train_dataset = self.dataset.iloc[0 : self.train_size * batch_size + time_steps]
        self.test_dataset = self.dataset.iloc[self.train_size * batch_size : (self.train_size + self.test_size) * batch_size + time_steps]
        self.train_labels = self.labels[0 : self.train_size * batch_size + time_steps]
        self.test_labels = self.labels[self.train_size * batch_size : (self.train_size + self.test_size) * batch_size + time_steps]
        self.train_diff = self.price_diff[0 : self.train_size * batch_size + time_steps]
        self.test_diff = self.price_diff[self.train_size * batch_size : (self.train_size + self.test_size) * batch_size + time_steps]
        
        self.train_cursor = 0
        self.test_cursor = 0
        self.use_weight = use_weight

        #print (self.train_dataset)
        print ("------------- test -----------")
        #print (self.test_dataset)
        print("Training set size: ", len(self.train_dataset)," Test set size: ", len(self.test_dataset))
        print("Batch Size: ", batch_size, "Train Batch Num: ", self.train_size,"Test Batch Num: ", self.test_size)
        print("Train interval: ",0, self.train_size * batch_size + time_steps)
        print("Test interval: ",self.train_size * batch_size, (self.train_size + self.test_size) * batch_size + time_steps)

    def reset(self):
        self.train_cursor = 0
        self.test_cursor = 0
    def _next_batch(self, b_index, isTraining = True):
        X_batch, y_batch, z_batch = [],[], []
        if isTraining == True:
            datasource = self.train_dataset
            labels = self.train_labels
            diff = self.train_diff
        else:
            datasource = self.test_dataset
            labels = self.test_labels
            diff = self.test_diff
        for i in range(self.batch_size):
            '''data_series_x = datasource['trend'].values[b_index + i : b_index + i + self.time_steps]
            X_batch.append(weight_assign(data_series_x, self.use_weight, self.fine_grained))#time-weighted processing
            y_batch.append(datasource['trend'].values[b_index + i + self.time_steps])
            z_batch.append(datasource['diff'].values[b_index + i + self.time_steps - 1])'''
            data_series_x = datasource.values[b_index + i : b_index + i + self.time_steps]
            X_batch.append(data_series_x)
            y_batch.append(labels.values[b_index + i + self.time_steps])
            z_batch.append(diff.values[b_index + i + self.time_steps]) #涨跌幅 (应该往后推一天，即与预测的值的差值)
            #input size changed to the total factors' number
            self.input_size = np.shape(datasource)[1]
        return np.array(X_batch).reshape(self.batch_size, self.time_steps, self.input_size),np.array(y_batch), np.array(z_batch)

    def next_batch(self, is_training=True):
        if is_training:
            xs, ys, zs = self._next_batch(self.train_cursor * self.batch_size, is_training) #rolling use
            self.train_cursor = (self.train_cursor + 1) % (self.train_size)
        else:
            xs, ys, zs = self._next_batch(self.test_cursor * self.batch_size, is_training)
            self.test_cursor = (self.test_cursor + 1) % self.test_size
        dic = {}
        dic['X'] = xs
        dic['y'] = ys
        dic['z'] = zs
        return dic


# yapf: disable
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
FLAGS = None

"""Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
def variable_summaries(var):    
    with tf.name_scope('summaries2'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

''' time recorder '''
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


############################
###### contruction phase ###
############################
def train(dataset, FLAGS):
    tf.reset_default_graph()

    # how long will a span cover, e.g. 20 days (4 tradable weeks)
    TIME_SPAN = FLAGS.time_steps # 4weeks, 1month
    TRAIN_RATIO = 0.9#0.8
    BATCH_SIZE = FLAGS.batch_size
    n_neurons = FLAGS.n_neurons #250
    n_steps = TIME_SPAN
    n_input = FLAGS.input_dim #5, 1 week's data as feature
    n_output = 3#5 #5 class
    n_layers = FLAGS.n_layers #5#10 #5 #3
    learning_rate = FLAGS.learning_rate #0.0005 # # 0.02 # 0.005
    print(FLAGS)
    
    with tf.name_scope('input'):
        X = tf.placeholder(tf.float32, [None, n_steps, n_input], name='X-input')
        y = tf.placeholder(tf.int32, [None], name='y-input') # every instance relates to 1 label
        #z = tf.placeholder(tf.float32, [None], name='price-input')
    
    with tf.variable_scope("lstm_layers", initializer=tf.contrib.layers.variance_scaling_initializer()):
        # define network, 3 LSTM layer now, use tf.tanh as activation function, use peephole
        #lstm_cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, use_peepholes=False)
        #lstm_cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, activation=tf.nn.relu, use_peepholes=False)
        stacked_cells = []
        for i in range(n_layers):
            stacked_cells.append(tf.contrib.rnn.LSTMCell(num_units=n_neurons))
        cells = tf.contrib.rnn.MultiRNNCell(stacked_cells)
        rnn_outputs, states = tf.nn.dynamic_rnn(cells, X, dtype=tf.float32)

    tf.summary.histogram('lstm_outputs',rnn_outputs) # new_h, every output
    #tf.summary.histogram('lstm_states', states)  # state is just the tuple(new_c, new_h)

    states = states[-1][1] #retrieve the last layer's state tuple and only need last output/hypothesis states, omit memory cell
    #print("Final STATES: ", states)
    tf.summary.histogram('lstm_cell_states', states)
    fc_layer = fully_connected(states, n_output, activation_fn=None)
    print ("label: ",y)
    print ("logits: ", fc_layer)
    #print ("price: ",z)
    with tf.name_scope('cross_entropy'):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=fc_layer)
        #print("Cross-entropy", xentropy)   #[batch_size, n_steps], float32
        # define loss function & optimize method
        with tf.name_scope('total_loss'):
            loss = tf.reduce_mean(xentropy)
            #print("LOSS:", loss)
    tf.summary.scalar('xentropy_mean', loss)

    with tf.name_scope('training'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss)

    # measurement
    with tf.name_scope('accuracy'):
        correct = tf.nn.in_top_k(fc_layer, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    tf.summary.scalar('accuracy_mean',accuracy)

    # rate of return, get all classes: if up or flat, add difference else add 0 
    with tf.name_scope("ROR"):
        classes = tf.argmax(fc_layer, axis=1)
        
    #########################################
    ############# Training Phase    #########
    #########################################
    n_epoch = FLAGS.n_epoch
    
    # yapf: enable
    ######### Start Training ########################
    start_time = time.time()
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        # Merge all the summaries and write them out to log dir
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(FLAGS.log_dir + '/train', sess.graph)
        test_writer = tf.summary.FileWriter(FLAGS.log_dir + '/test')    
    
        avg_train, avg_test = [0], [0]
        sum_train, sum_test =0., 0. #for all iterations
        train_res, test_res = [], []
        return_train = 0.
        
        init.run()
        for j in range(n_epoch):
            dataset.reset()
            best_train_acc, sum_train_acc = 0., 0.
            best_test_acc, sum_test_acc = 0., 0.
            test_threshold = dataset.train_size
            return_train_epoch = 0.
            print("---- Epoch ",j+1)
            for i in range(dataset.segment_num):    
                ### Testing after training
                if i > test_threshold:
                    # Record summaries and test-set accuracy
                    data = dataset.next_batch(is_training=False)
                    summary, acc_test = sess.run([merged, accuracy], feed_dict={X: data['X'], y: data['y']})
                    test_writer.add_summary(summary, i+j*FLAGS.max_steps)
                    #print('Test Accuracy %s: %s' % (i, acc_test))
                    if acc_test > best_test_acc:
                        best_test_acc = acc_test
                    sum_test_acc += acc_test
                    sum_test += acc_test
                    avg_test.append(sum_test/(len(avg_test)+1))
                    #test_res.append(acc_test)
                # Record train set summaries and train
                if i % 100 == 99: # Record execution stats    
                    data = dataset.next_batch()
                    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                    run_metadata = tf.RunMetadata()
                    summary, acc_train, _ = sess.run([merged, accuracy, training_op],
                    #summary, _, acc, fp, fl, cr, out, st = sess.run([merged, training_op, accuracy,final_predict,final_label,correct,rnn_outputs, states],
                                          feed_dict={X: data['X'], y: data['y']},
                                          options=run_options,
                                          run_metadata=run_metadata)
                    #train_writer.add_run_metadata(run_metadata, 'step%03d' % (i+j*FLAGS.max_steps))
                    train_writer.add_summary(summary, i+j*FLAGS.max_steps)
                    #print('Adding run metadata for %d, Acc: %s' % (i, acc_train))
                    sum_train_acc += acc_train
                    # add average training accuracy
                    sum_train += acc_train
                    avg_train.append(sum_train/(len(avg_train)+1))
                    #train_res.append(acc_train)
                    
                else:
                    data = dataset.next_batch()
                    summary, _, acc_train, clz = sess.run([merged, training_op, accuracy, classes],
                                                          feed_dict={X: data['X'], y: data['y']})
                    #print ("classes: ", clz)
                    z = data['z']
                    for j in range(len(z)):
                        if clz[j] > 0:
                            return_train_epoch += z[j] #add difference
                    
                    train_writer.add_summary(summary, i+j*FLAGS.max_steps)
                    if acc_train > best_train_acc:
                        best_train_acc = acc_train
                    sum_train_acc += acc_train
                    # add average training accuracy
                    sum_train += acc_train
                    avg_train.append(sum_train/(len(avg_train)+1))
                    #train_res.append(acc_train)
            return_train += return_train_epoch   
            print(" ========= EPOCH %d STATISTICS============" % (j))
            print("   BEST Train Accuracy:", best_train_acc, " AVERAGE Train Accuracy:", sum_train_acc/dataset.train_size)
            print("   BEST Test Accuracy:", best_test_acc, " AVERAGE Test Accuracy:", sum_test_acc/dataset.test_size)
            print(" ========= ========= ========= =========")
        train_writer.close()
        test_writer.close()
        #x_test = [i for i in range(0, FLAGS.max_steps*n_epoch, test_freq-1)]
        size = len(avg_train)
        
        print("-- AVG ROR: ", return_train / n_epoch)
        print("==== FINAL TRAINING ACC: ", avg_train[-1])
        print("==== FINAL TESTING ACC: ", avg_test[-1])
        print(" ========= ========= ========= =========")
        #plt.title("Average Accuracy of the Model", fontsize=34)
        #plt.plot(avg_train, markersize=12, linewidth=3, label="Train")
        #plt.plot(x_test[: len(avg_test)], avg_test, markersize=12, linewidth=3, label="Test")
        #plt.legend(loc="upper left")
        #plt.xlabel("Steps", fontsize=16)
        #plt.ylabel("Average Accuracy", fontsize=16)
        #plt.show()
        return avg_train[-1], avg_test[-1]
        
def main(_):
    if tf.gfile.Exists(FLAGS.log_dir):
        tf.gfile.DeleteRecursively(FLAGS.log_dir)
    tf.gfile.MakeDirs(FLAGS.log_dir)
    #filename = '../data/dataset/close_weekly-2007-2017.csv'
    filename = '../data/dataset/close_2002-2017.csv'
    #filename = '../data/dataset/Dow Jones Industrial Average.csv'
    #filename = '../data/dataset/S&P 500.csv'
    dataset = BatchGenerator(filename,  FLAGS.batch_size, FLAGS.train_ratio,FLAGS.time_steps, FLAGS.input_dim, column = 1, retrace = FLAGS.retrace, fold_i=0, use_weight=FLAGS.use_weight)
    train(dataset, FLAGS)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('--max_steps', type=int, default=100,
                          help='Number of steps to run trainer.')
    parser.add_argument('--learning_rate', type=float, default=0.001,
                          help='Initial learning rate')
    parser.add_argument('--batch_size', type=int, default=20,
                          help='number of instances in a batch')
    parser.add_argument('--time_steps', type=int, default=4,
                          help='Number of time steps.')
    parser.add_argument('--input_dim', type=int, default=48, #dimension of factors
                          help='Dimension of inputs.')
    parser.add_argument('--n_neurons', type=int, default=150,
                          help='Number of neurons.')
    parser.add_argument('--n_layers', type=int, default=3,
                          help='Number of lstm layers.')
    parser.add_argument('--train_ratio', type=float, default=0.8,
                          help='Keep probability for training dropout.')                      
    parser.add_argument('--dropout', type=float, default=0.9,
                          help='Keep probability for training dropout.')
    parser.add_argument('--retrace', type=float, default=0.618,
                          help='Retracement of wave.')
    parser.add_argument('--use_weight', type=float, default=0.0,
                          help='Whether use time-weighted function.')               
    parser.add_argument('--n_epoch', type=int, default=50,
                          help='Epoch Number.')
    parser.add_argument('--no_retrace', type=bool, default=False,
                          help='Whether use retracement.')
    parser.add_argument('--fine_grained', type=bool, default=False,
                          help='Use fine-grained weight function.')
                          
    parser.add_argument(
          '--data_dir',
          type=str,
          default='./input_data',
          help='Directory for storing input data')
    parser.add_argument(
          '--log_dir',
          type=str,
          default='./logs',
          help='Summaries log directory')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

../data/000300.SH/000300.SH.csv ===============
Factors Shape: (2671, 10) (2671, 38)
----- start:  0
---- Trend Distribution Check --------
0.0     960
1.0     558
2.0    1153
dtype: int64
input data shape:  (2671, 48)
input label shape:  (2671,)
------------- test -----------
Training set size:  2124  Test set size:  544
Batch Size:  20 Train Batch Num:  106 Test Batch Num:  27
Train interval:  0 2124
Test interval:  2120 2664
Namespace(batch_size=20, data_dir='./input_data', dropout=0.9, fine_grained=False, input_dim=48, learning_rate=0.001, log_dir='./logs', max_steps=100, n_epoch=50, n_layers=3, n_neurons=150, no_retrace=False, retrace=0.618, time_steps=4, train_ratio=0.8, use_weight=0.0)
label:  Tensor("input/y-input:0", shape=(?,), dtype=int32)
logits:  Tensor("fully_connected/BiasAdd:0", shape=(?, 3), dtype=float32)
---- Epoch  1
 ========= EPOCH 19 STATISTICS============
   BEST Train Accuracy: 1.0  AVERAGE Train Accuracy: 0.51415094431
   BEST Test Accuracy: 0.8  AVERAGE Test 

 ========= EPOCH 19 STATISTICS============
   BEST Train Accuracy: 1.0  AVERAGE Train Accuracy: 0.693867923832
   BEST Test Accuracy: 0.85  AVERAGE Test Accuracy: 0.450000000497
 ========= ========= ========= =========
---- Epoch  34
 ========= EPOCH 19 STATISTICS============
   BEST Train Accuracy: 1.0  AVERAGE Train Accuracy: 0.704245282478
   BEST Test Accuracy: 0.85  AVERAGE Test Accuracy: 0.461111111084
 ========= ========= ========= =========
---- Epoch  35
 ========= EPOCH 19 STATISTICS============
   BEST Train Accuracy: 1.0  AVERAGE Train Accuracy: 0.712264151365
   BEST Test Accuracy: 0.85  AVERAGE Test Accuracy: 0.459259259756
 ========= ========= ========= =========
---- Epoch  36
 ========= EPOCH 19 STATISTICS============
   BEST Train Accuracy: 1.0  AVERAGE Train Accuracy: 0.70141509467
   BEST Test Accuracy: 0.85  AVERAGE Test Accuracy: 0.448148146548
 ========= ========= ========= =========
---- Epoch  37
 ========= EPOCH 19 STATISTICS============
   BEST Train Accuracy

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [45]:
_data_prepare('../data/dataset/close_weekly-2007-2017.csv')

1 ===
../data/dataset/close_weekly-2007-2017.csv
----- start:  0
---- Trend Distribution Check --------
0    201
1    123
2    210
Name: trend, dtype: int64
2017-12-03 18:07:25  ------ Complete Data Preparation
total data:  534


,close,mmt,diff,label,trend
date,,,,,
2007-01-05,2678.526500,0.000000,77.233300,1,1
2007-01-12,2755.759800,0.028834,41.001800,2,1
2007-01-19,2796.761600,0.014879,122.715600,2,1
2007-01-26,2919.477200,0.043878,-95.316400,2,1
2007-02-02,2824.160800,-0.032648,-129.655400,0,1
2007-02-09,2694.505400,-0.045909,212.618800,0,2
2007-02-16,2907.124200,0.078908,-42.688400,2,2
2007-03-02,2864.435800,-0.014684,13.164200,0,2
2007-03-09,2877.600000,0.004596,64.044200,1,2


In [80]:
dataset = BatchGenerator('../data/dataset/close_weekly-2007-2017.csv',batch_size=20, train_ratio=0.9, time_steps=4, input_size=1)
dataset.next_batch()

../data/000300.SH/000300.SH.csv ===============
Factors Shape: (2671, 10) (2671, 38)
----- start:  0
---- Trend Distribution Check --------
0.0     960
1.0     558
2.0    1153
dtype: int64
input data shape:  (2671, 48)
input label shape:  (2671,)
------------- test -----------
Training set size:  2384  Test set size:  284
Batch Size:  20 Train Batch Num:  119 Test Batch Num:  14
Train interval:  0 2384
Test interval:  2380 2664


{'X': array([[[  1.00000000e+03,   9.94770000e+02,   9.94770000e+02, ...,
            0.00000000e+00,  -6.29077355e+06,   0.00000000e+00],
         [  9.82790000e+02,   9.81580000e+02,   9.97320000e+02, ...,
            0.00000000e+00,  -1.80735899e+06,   9.60371624e-01],
         [  9.92560000e+02,   9.93330000e+02,   9.93790000e+02, ...,
            0.00000000e+00,  -6.55305011e+06,   8.65405808e-01],
         [  9.83170000e+02,   9.83040000e+02,   9.95710000e+02, ...,
            0.00000000e+00,  -6.13392264e+06,   1.05168467e+00]],
 
        [[  9.82790000e+02,   9.81580000e+02,   9.97320000e+02, ...,
            0.00000000e+00,  -1.80735899e+06,   9.60371624e-01],
         [  9.92560000e+02,   9.93330000e+02,   9.93790000e+02, ...,
            0.00000000e+00,  -6.55305011e+06,   8.65405808e-01],
         [  9.83170000e+02,   9.83040000e+02,   9.95710000e+02, ...,
            0.00000000e+00,  -6.13392264e+06,   1.05168467e+00],
         [  9.83960000e+02,   9.83760000e+02,   9.9396

In [76]:
from sklearn import preprocessing
def data_prepare(retrace = 0.618):
    # prepare data
    baseDir = '../data/'
    stockCodes = ['000300.SH']
    i = 0
    startYear = 2005
    number =11
    df = readWSDFile(baseDir, stockCodes[i], startYear, number)
    dfi = readWSDIndexFile(baseDir, stockCodes[i], startYear, number)
    allDF = pd.concat([df, dfi], axis=1)
    sample_num = np.shape(df)[0]
    labelDF = pd.Series(np.zeros(sample_num), dtype=np.int32)  
    print ("Factors Shape:", np.shape(df), np.shape(dfi))
    
    # 求出 trend
    price = df['close']
    start = 0
    while price[start] > price[start+1]:
        labelDF[start] = 1 #flat
        start +=1
    print("----- start: ",start)
    #find peak, find trough, calculate retracement and label trend accordingly
    i = start
    while i < sample_num - 1:
        cursor = i
        while cursor < sample_num - 1 and price[cursor] <= price[cursor+1]:
            cursor += 1
        peak = cursor
        while cursor < sample_num - 1 and price[cursor] >= price[cursor+1]:
            cursor += 1
        trough = cursor
        retracement = (price[peak] - price[trough]) / (price[peak] - price[i])
        mark = 1 # flat
        if retracement < retrace:
            mark = 2 # UP
        elif retracement > 1 + retrace:
            mark = 0 # DOWN
        for k in range(i, cursor+1):
            labelDF[k] = mark
        i = cursor

    # make a deep copy of Price Difference before normalizing
    priceDF = allDF['change'].copy(deep=True)
    # normalize
    #scaler = preprocessing.MinMaxScaler()
    #input_data = scaler.fit_transform(allDF)
    
    print ("input data shape: ", np.shape(allDF)) #  days *  factors
    print ("input label shape: ", np.shape(labelDF))
   
    return allDF, labelDF, priceDF
data_prepare()

../data/000300.SH/000300.SH.csv ===============
Factors Shape: (2671, 10) (2671, 38)
----- start:  0
input data shape:  (2671, 48)
input label shape:  (2671,)


(            pre_close     open     high      low    close  change  chg_range  \
 2005-01-04    1000.00   994.77   994.77   980.66   982.79  -17.21    -1.7206   
 2005-01-05     982.79   981.58   997.32   979.88   992.56    9.77     0.9941   
 2005-01-06     992.56   993.33   993.79   980.33   983.17   -9.39    -0.9460   
 2005-01-07     983.17   983.04   995.71   979.81   983.96    0.78     0.0797   
 2005-01-10     983.96   983.76   993.96   979.79   993.88    9.92     1.0083   
 2005-01-11     993.88   994.19   999.55   991.09   997.14    3.26     0.3276   
 2005-01-12     997.14   996.65   996.98   989.26   996.75   -0.39    -0.0388   
 2005-01-13     996.75   996.08   999.47   992.70   996.88    0.13     0.0129   
 2005-01-14     996.88   996.62  1006.46   987.23   988.31   -8.57    -0.8598   
 2005-01-17     988.31   979.11   981.52   965.08   967.45  -20.85    -2.1101   
 2005-01-18     967.45   967.37   974.87   960.29   974.69    7.24     0.7480   
 2005-01-19     974.69   974

In [77]:
# TEST Pandas DEEP copy
d = {'one' : pd.Series([1., 2., 3.], index=['a', 'b', 'c']),'two' : pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])}
s = d['one']
s2 = d['one'].copy(deep=True)
#print(s)
d['one'][1] = 2.33333
#print(s)
#print(s2)

In [ ]:
'''
日志
12.04 LSTM 输入重塑，读入技术指标及行情数据，data_prepare单元测试通过
12.05 next_batch输出符合预期, trend是timestep之后的标签，price_diff是timestep之后与未来一天的价差。预测准确度在65%，收益率40%
    
'''